In [51]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
from config import (
    OPENROUTER_DEEPSEEK_MODEL, 
    OPENROUTER_BASE_URL, 
    DEEPSEEK_MODEL, 
    DEEPSEEK_BASE_URL,
    EMBEDDING_MODEL,
    TUZI_BASE_URL
)

# 加载.env文件中的环境变量
load_dotenv()

# 从环境变量获取OpenRouter API密钥
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")

# 初始化ChatOpenAI模型，指定base_url为OpenRouter的API端点
llm = ChatOpenAI(
    model=DEEPSEEK_MODEL,  # 例如 "openai/gpt-3.5-turbo"
    api_key=deepseek_api_key,
    base_url=DEEPSEEK_BASE_URL
)

# 使用模型进行聊天
response = llm.invoke("你好，介绍一下自己。")
print(response)

content='你好！我是 **DeepSeek Chat**，由深度求索公司（DeepSeek）研发的智能AI助手。我可以帮助你解答各种问题，包括学习、工作、编程、写作、翻译、生活建议等。  \n\n### **我的特点：**  \n🔹 **免费使用**：目前不收费，你可以随时向我提问！  \n🔹 **知识丰富**：我的知识截止到 **2024年7月**，能提供较新的信息。  \n🔹 **超长上下文**：支持 **128K** 上下文，可以处理超长文档和复杂对话。  \n🔹 **文件阅读**：可以解析 **PDF、Word、Excel、PPT、TXT** 等文件内容。  \n🔹 **逻辑与创造力兼备**：既能帮你写代码、分析数据，也能创作诗歌、故事和文案。  \n\n### **我能帮你做什么？**  \n📚 **学习辅导**：解题思路、论文润色、语言学习  \n💼 **工作效率**：写邮件、做PPT、整理会议纪要  \n💡 **编程助手**：代码调试、算法优化、技术解答  \n📝 **写作创意**：小说、文案、广告语、社交媒体帖子  \n🌍 **旅行 & 生活**：推荐攻略、美食、健康建议  \n\n如果你有任何问题，尽管问我吧！😊' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 278, 'prompt_tokens': 8, 'total_tokens': 286, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 8}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_3d5141a69a_prod0225', 'id': '9677dec2-87b4-48fa-abce-0ac07e2b5337', 'finish_reason': 'stop', 'logprobs': None}

In [52]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain_openai import OpenAIEmbeddings 

# 2. Prompts - 创建提示模板
prompt_template = PromptTemplate(
    input_variables=["product"],
    template="请为{product}写一个简短的广告语。"
)

# 3. Chains - 创建简单的链
simple_chain = LLMChain(
    llm=llm,
    prompt=prompt_template
)

# 4. Memory - 创建记忆
memory = ConversationBufferMemory(return_messages=True)

# 5. Tools - 加载工具
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# 6. Agents - 初始化代理
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)

# 7. Indexes - 创建索引(检索增强生成RAG的基础)
loader = TextLoader("example_data.txt")
embedding_api_key = os.getenv("TUZI_API_KEY")
embeddings = OpenAIEmbeddings(
    model=EMBEDDING_MODEL,
    openai_api_key=embedding_api_key,
    openai_api_base=TUZI_BASE_URL,
    tiktoken_enabled=True
)
index = VectorstoreIndexCreator(embedding=embeddings).from_loaders([loader])

/Users/childhoodandy/.pyenv/versions/3.11.3/lib/python3.11/site-packages/langchain/indexes/vectorstore.py:171: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


In [53]:
# 使用简单的链
ad_result = simple_chain.run(product="智能手表")
print(f"广告语生成结果: {ad_result}")

广告语生成结果: **“腕间智慧，健康随行——你的24小时智能生活管家！”**  

（简洁突出核心功能：智能操控+健康监测，同时强调便捷与全天候陪伴感。）


In [55]:
# 使用代理(可以使用工具和记忆)
agent_result = agent.run("谁是阿尔伯特·爱因斯坦？他的质能方程是什么？")
print(f"代理回答结果: {agent_result}")



> Entering new AgentExecutor chain...
Thought: 我需要先了解阿尔伯特·爱因斯坦是谁，然后查找他的质能方程。

Action: wikipedia  
Action Input: 阿尔伯特·爱因斯坦  

Observation: Page: Honkai Impact 3rd
Summary: Honkai Impact 3rd is a free-to-play 3D action role-playing game developed and published by miHoYo (with publishing outside mainland China under Cognosphere, trading as HoYoverse). It is the spiritual successor to Houkai Gakuen 2, using many characters from the previous title in a separate story. The game is notable for incorporating a variety of genres, from hack and slash and social simulation, to elements of bullet hell, platforming, shoot 'em up and dungeon crawling across multiple single and multiplayer modes. It features gacha mechanics. It was first released on mobile devices and later ported to Microsoft Windows.
A massive expansion of the game, titled Honkai Impact 3rd Part 2 was released on February 29, 2024. Built upon the existing game, the update marks the start of the second major story arc of the game,

In [54]:

# 使用索引进行检索问答
query_result = index.query("文档中最重要的主题是什么？",llm=llm)
print(f"检索问答结果: {query_result}")

检索问答结果: 根据提供的文档内容，最重要的主题可以归纳为以下几个方面：

1. **人工智能的定义与分类**  
   文档首先明确了AI的核心概念（模仿人类智能的计算机系统）及其三大类型（狭义AI、通用AI、超级AI），这是理解整个领域的基础框架。

2. **核心技术（机器学习与深度学习）**  
   作为当前AI发展的核心驱动力，文档详细介绍了机器学习的三大范式（监督/无监督/强化学习）和深度学习的典型模型架构（CNN、RNN、变换器），这些技术支撑了绝大多数AI应用。

3. **跨领域应用**  
   文档列举了AI在医疗、金融、交通等关键领域的实际价值，凸显了其社会影响力。这些应用案例直接体现了AI技术的现实意义。

4. **伦理挑战**  
   特别强调了AI发展伴随的隐私、偏见、就业影响等伦理问题，反映出技术与社会责任的平衡是当前的重要议题。

**最关键主题**：  
**"机器学习与深度学习技术"**是文档中最核心的内容，因为它们是实现所有AI应用的基础方法论，同时其发展也直接关联到伦理挑战和未来趋势的讨论。其他主题（如分类、应用）均围绕这些核心技术展开。
